In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Training**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/supportivTask/intern_screening_dataset.csv')

In [ ]:
df

,question,answer
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...
1,What is (are) Glaucoma ?,The optic nerve is a bundle of more than 1 mil...
2,What is (are) Glaucoma ?,Open-angle glaucoma is the most common form of...
3,Who is at risk for Glaucoma? ?,Anyone can develop glaucoma. Some people are a...
4,How to prevent Glaucoma ?,"At this time, we do not know how to prevent gl..."
...,...,...
16401,What is (are) Diabetic Neuropathies: The Nerve...,Autonomic neuropathy affects the nerves that c...
16402,What is (are) Diabetic Neuropathies: The Nerve...,"Proximal neuropathy, sometimes called lumbosac..."
16403,What is (are) Diabetic Neuropathies: The Nerve...,Focal neuropathy appears suddenly and affects ...
16404,How to prevent Diabetic Neuropathies: The Nerv...,The best way to prevent neuropathy is to keep ...


In [ ]:
df = df.dropna()

In [ ]:
df.shape

(16401, 2)

In [ ]:
df.drop_duplicates(subset=['question'], keep='first', inplace=True)
df.reset_index(drop=True, inplace=True)

<ipython-input-9-5da7f6bd2b2f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['question'], keep='first', inplace=True)


In [ ]:
df.shape

(14976, 2)

In [ ]:
# df = df.iloc[:2500]

In [ ]:
def df_to_txt(dataframe, file_name):
    with open(file_name, 'w') as file:
        for index, row in dataframe.iterrows():
          file.write(f"{row['question']}")
          file.write(f"{row['answer']}\n")

# Write the DataFrame to a text file
df_to_txt(df, 'input.txt')

In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print(text[0:1000])

What is (are) Glaucoma ?Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. The most common form of the disease is open-angle glaucoma. With early treatment, you can often protect your eyes against serious vision loss. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.)  See this graphic for a quick overview of glaucoma, including how many people it affects, whos at risk, what to do if you have it, and how to learn more.  See a glossary of glaucoma terms.
Who is at risk for Glaucoma? ?Anyone can develop glaucoma. Some people are at higher risk than others. They include - African-Americans over age 40  - everyone over age 60, especially Hispanics/Latinos  - people with a family history of glaucoma. African-Americans over age 40 everyone over age 60, especially Hispanics/Latinos people with a f

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [ ]:
def load_dataset(file_path, tokenizer, block_size = 512):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator


def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [ ]:
train_file_path = "/content/input.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/result'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 1
save_steps = 1000

In [ ]:
!pip install transformers[torch] -q
!pip install accelerate -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.7 MB/s eta 0:00:00


In [ ]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

Step,Training Loss
500,1.801500
1000,1.620700


# QNA


In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [ ]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length):
    model_path = "/content/drive/MyDrive/result"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    return (tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [ ]:
#test1
sequence = 'What does High blood pressure cause?'
max_len = int(128)
text = generate_text(sequence, max_len)
print(text)

What does High blood pressure cause? It can affect anything from heart to liver function, including blood clotting. High blood pressure may cause weakness and/or a feeling of weightlessness in your arms, legs, or neck (hypertelorism). High blood pressure can lead to heart problems, a lack of energy in your blood vessels, loss of coordination (congestive heart failure), a blockage in the heart muscle, or shortness of breath. High blood pressure can make your heart unable to respond to pumping blood effectively. Your doctor will diagnose High blood pressure with a heart examination.    Low-dose medications can


In [ ]:
#test2
sequence = 'Does glaucoma affect something?'
max_len = int(128)
text = generate_text(sequence, max_len)
print(text)

Does glaucoma affect something? A diagnosis of glaucoma likely isn't a sign of disease, but may suggest an underlying condition. Your doctor may:

come to a diagnosis of glaucoma with a detailed description of the condition and the cause The doctor may include details about the signs and symptoms of the condition

find ways to relieve symptoms and the possible course of the disorder

find ways to raise awareness of glaucoma and increase awareness of the problems with glaucoma

come to a diagnosis of glaucoma with a detailed description of the condition and the cause The doctor
